## Value Iteration Algorithm

**Questions:**

1) Write the value iteration algorithm in Python and find the optimal state value for all states. Assume a discounting factor 𝛾 = 0.8.

2) Write an algorithm in Python which computes the optimal policy 𝜋 for all states.


### **1. Preparatory Processes**

In [ ]:
import numpy as np

# Define the rows and columns
n_rows = 3
n_cols = 5

# Define the terminal states and the walls/impossible states
terminals = [(0, 4), (2, 3)]
walls = [(0, 3), (1, 1)]

# Define the possible actions at each state
actions = ["UP", "RIGHT", "DOWN", "LEFT"]

# Define the row and column changes from each state to new states depending on actions taken
row_change = [-1, 0, 1, 0] # up, right, down, left
col_change = [0, 1, 0, -1] # up, right, down, left

# Define the motion model
motion_probs = {
    'UP':    {'UP': 0.8, 'RIGHT': 0.1, 'DOWN': 0.0, 'LEFT': 0.1},
    'RIGHT': {'UP': 0.1, 'RIGHT': 0.8, 'DOWN': 0.1, 'LEFT': 0.0},
    'DOWN':  {'UP': 0.0, 'RIGHT': 0.1, 'DOWN': 0.8, 'LEFT': 0.1},
    'LEFT':  {'UP': 0.1, 'RIGHT': 0.0, 'DOWN': 0.1, 'LEFT': 0.8}
}

# Definition and initialization of the reward function
R = np.full((n_rows, n_cols), -0.1)
R[0, 4] = 1 # Reward at terminal state I (Goal state)
R[2, 3] = -1 # Reward at terminal state II (Bad state)

# Define the number of episodes (or iterations)
episode = 100

# Define the discount factor
gamma = 0.8

# Define threshold
threshold = 1e-6

# Definition and initialization of the value for each state.
v = np.zeros((n_rows, n_cols))

### **2. Now it's time to implement the value iteration algorithm using the Bellman equation.**


*   ### Bellman equation: **V(s) = R(s) + γmax(∑(P(s'|s,a)* V(s'))**



In [ ]:
# Value iteration algorithm
for e in range(episode):
  print("---------------------------------------------------\n")
  print(f"Episode: {e}\n")
  print("---------------------------------------------------\n")
  delta = 0
  for row in range(n_rows):
    for col in range(n_cols):
        if (row, col) == terminals[0]:
          v[row, col] = 1
          print(f"Value for state v {row, col} = {v[row,col]}")
          continue

        elif (row, col) == terminals[1]:
          v[row, col] = -1
          print(f"Value for state v {row, col} = {v[row,col]}")
          continue

        elif (row, col) in walls:
          print(f"Value for blocked state v {row, col} is Nil")
          continue

        else:
          p_next_states = [] # Initialize list for plausible next states
          v_next_states = [] # Initialize list for the values of the plausible next states
          for i, a in enumerate(actions):
              next_row = row + row_change[i]
              next_col = col + col_change[i]
              if next_row < 0 or next_row >= n_rows or next_col < 0 or next_col >= n_cols or (next_row, next_col) in walls: # Checking to ensure boundaries of the environment is not violated.
                  next_state = (row, col)
                  p_next_states.append(next_state)
                  v_next_states.append(v[next_state])
              else:
                  next_state = (next_row, next_col)
                  p_next_states.append(next_state)
                  v_next_states.append(v[next_state])

          v_check = []  # Initialize empty list for the value of state after taking an action from up, right, down, left.
          old_v = v[row, col] # Define old value to check for convergence
          for a in actions:
            act_prob = list(motion_probs[a].values())
            temp_res = [act_prob[i] * v_next_states[i] for i in range(len(v_next_states))] # (P(s'|s,a)* V(s')
            sum_temp_res = sum(temp_res) # ∑(P(s'|s,a)* V(s')
            v_check.append(sum_temp_res)
          v_selected = max(v_check)
          v[row, col] = R[row, col] + gamma * v_selected # R(s) + γmax(∑(P(s'|s,a)* V(s'))
          print(f"New value for state v{row,col} = {v[row,col]:.5f}")
          delta = max(delta, abs(old_v - v[row,col]))

  if delta < threshold:
    print("----------------------------------------------\n")
    print(f"\nFinal value of v is: \n\n {v}")
    break
  else:
    continue # go to next episode

---------------------------------------------------

Episode: 0

---------------------------------------------------

New value for state v(0, 0) = -0.10000
New value for state v(0, 1) = -0.10000
New value for state v(0, 2) = -0.10000
Value for blocked state v (0, 3) is Nil
Value for state v (0, 4) = 1.0
New value for state v(1, 0) = -0.10000
Value for blocked state v (1, 1) is Nil
New value for state v(1, 2) = -0.10000
New value for state v(1, 3) = -0.10000
New value for state v(1, 4) = 0.53200
New value for state v(2, 0) = -0.10000
New value for state v(2, 1) = -0.10000
New value for state v(2, 2) = -0.10800
Value for state v (2, 3) = -1.0
New value for state v(2, 4) = 0.16048
---------------------------------------------------

Episode: 1

---------------------------------------------------

New value for state v(0, 0) = -0.18000
New value for state v(0, 1) = -0.18000
New value for state v(0, 2) = -0.18000
Value for blocked state v (0, 3) is Nil
Value for state v (0, 4) = 1.0
New va

### **3. After the value iteration converges, the optimal policy for each state can be obtained.**
* This is implemented by selecting the action which yields the maximum value in each state.
* ### **π\*(s) = argmax(∑(P(s'|s,a)* V(s'))**

In [ ]:
# Deriving the optimal policy

# initialize the opt_policy with a series of texts
opt_policy = [['undecided', 'undecided', 'undecided', 'None', 'Terminal'],
              ['undecided', 'None', 'undecided', 'undecided', 'undecided'],
              ['undecided', 'undecided', 'undecided', 'Terminal', 'undecided']]


for row in range(n_rows):
    for col in range(n_cols):
        if (row, col) == terminals[0]:
          v[row, col] = 1
          continue
        elif (row, col) == terminals[1]:
          v[row, col] = -1
          continue
        elif (row, col) in walls:
          continue
        else:
          p_next_states = []
          v_next_states = []
          for i, a in enumerate(actions):
              next_row = row + row_change[i]
              next_col = col + col_change[i]
              if next_row < 0 or next_row >= n_rows or next_col < 0 or next_col >= n_cols or (next_row, next_col) in walls:
                  next_state = (row, col)
                  p_next_states.append(next_state)
                  v_next_states.append(v[next_state])
              else:
                  next_state = (next_row, next_col)
                  p_next_states.append(next_state)
                  v_next_states.append(v[next_state])
          v_check = []
          for a in actions:
            act_prob = list(motion_probs[a].values())
            temp_res = [v_next_states[i] * act_prob[i] for i in range(len(v_next_states))]
            sum_temp_res = sum(temp_res)
            v_check.append(sum_temp_res) # ∑(P(s'|s,a)* V(s')
          max_index = np.argmax(v_check) # Extract the index of the highest value in that state: argmax(∑(P(s'|s,a)* V(s'))
          opt_policy[row][col] = actions[max_index] # Update the policy

print("The optimal policy for each state is as shown below: \n\n")
opt_policy

The optimal policy for each state is as shown below: 




[['RIGHT', 'RIGHT', 'DOWN', 'None', 'Terminal'],
 ['UP', 'None', 'RIGHT', 'RIGHT', 'UP'],
 ['RIGHT', 'RIGHT', 'UP', 'Terminal', 'UP']]

### **4a. From the optimal policy, we see that:**
* In state (0,0): Go right
* In state (0,1): Go right
* In state (0,2): Go down
* state (0,3) is a wall
* state (0,4) is the goal state (terminal)
* In state (1,0): Go up
* state (1,1) is a wall







### **4b. From the optimal policy, we see that:**
* In state (1,2): Go right
* In state (1,3): Go right
* In state (1,4): Go up
* In state (2,0): Go right
* In state (2,1): Go right
* In state (2,2): Go up
* state (2,3) is the bad state (terminal)
* In state (2, 4): Go up

### **Conclusion**

Hence if starting at state (2,1), in python indexing (1,0). To get to the goal i.e. (0, 4) in python indexing at the shortest time possible. The Agent would follow this sequence of actions:
* **`up -> right -> right -> down -> right -> right -> up`**